<a href="https://colab.research.google.com/github/gsilver321/project_voting/blob/main/voting_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np
import pandas as pd

! git clone https://github.com/gsilver321/project_voting

Cloning into 'project_voting'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 123 (delta 22), reused 4 (delta 4), pack-reused 83
Receiving objects: 100% (123/123), 123.56 MiB | 17.59 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Updating files: 100% (63/63), done.


In [32]:
tsdf = pd.read_csv("project_voting/data/county_data/0002_ts_nominal_county.csv", low_memory=False, encoding="latin-1") # provided time series
tsdf = tsdf.loc[tsdf["STATE"] == "Virginia"] # limit time series to Virginia

tsdf2 = pd.read_csv("project_voting/data/county_data/nhgis0001_ts_nominal_county.csv", low_memory=False, encoding="latin-1") # downloaded time series with additional demographic info
tsdf2 = tsdf2.loc[tsdf2["STATE"] == "Virginia"]

vdf = pd.read_csv('project_voting/data/voting_VA.csv')
votes = vdf.loc[:, ["year", "county_name", "party", "candidatevotes"]].groupby(["year", "county_name", "party"]).sum() # convert voting format to determine winners
votes

candidatevotes
year county_name party                      
2000 ACCOMACK    DEMOCRAT               5092
                 GREEN                   220
                 OTHER                   261
                 REPUBLICAN             6352
     ALBEMARLE   DEMOCRAT              16255
...                                      ...
2020 WYTHE       REPUBLICAN            11733
     YORK        DEMOCRAT              17683
                 LIBERTARIAN             680
                 OTHER                   187
                 REPUBLICAN            20241

[2596 rows x 1 columns]

In [33]:
win_arr = []

for year in votes.groupby(level=0): # loop through year
  for county in year[1].groupby(level=1): # loop through county
    county_votes = county[1]["candidatevotes"]
    for i, vote in enumerate(county_votes): # loop through votes
      # if current vote makes up the largest share of votes in that county for current year:
      if vote == county_votes.max():
        win_arr.append([year[0], county[0], county_votes.index[i][2]]) # make that party the winner

df_win = pd.DataFrame(win_arr, columns=["year", "county", "winner"])
df_win

,year,county,winner
0,2000,ACCOMACK,REPUBLICAN
1,2000,ALBEMARLE,REPUBLICAN
2,2000,ALEXANDRIA,DEMOCRAT
3,2000,ALLEGHANY,REPUBLICAN
4,2000,AMELIA,REPUBLICAN
...,...,...,...
773,2020,WILLIAMSBURG CITY,DEMOCRAT
774,2020,WINCHESTER CITY,DEMOCRAT
775,2020,WISE,REPUBLICAN
776,2020,WYTHE,REPUBLICAN


In [34]:
df_avg = pd.DataFrame(tsdf["COUNTY"])

# Start demographic time series at year 2000 since that's when voting data starts:
# num of 25+ y.o. college-educated citizens in Virginia counties over time
df_edu = tsdf.loc[:, ["B69AC2000", "B69AC125", "B69AC195"]]
df_edu = df_edu.apply(pd.to_numeric)

# num of 16+ y.o. workers in Virginia counties over time
df_work = tsdf.loc[:, ["B84AA2000", "B84AA125", "B84AA195"]]
df_work = df_work.apply(pd.to_numeric)

# num of foreign-born citizens in Virginia counties over time
df_foreign = tsdf.loc[:, ["AT5AB2000", "AT5AB125", "AT5AB195"]]
df_foreign = df_foreign.apply(pd.to_numeric)

# num of female citizens in Virginia counties over time
df_female = tsdf.loc[:, ["AV1AB2000", "AV1AB125", "AV1AB195"]]
df_female = df_female.apply(pd.to_numeric)

# per capita income in Virginia counties over time
df_income = tsdf.loc[:, ["BD5AA2000", "BD5AA125", "BD5AA195"]]
df_income = df_income.apply(pd.to_numeric)

# num of citizens below poverty line in Virginia counties over time
df_poverty = tsdf.loc[:, ["CL6AA2000", "CL6AA125", "CL6AA195"]]
df_poverty = df_poverty.apply(pd.to_numeric)

# num of citizens in rural areas in Virginia counties over time
df_rural = tsdf2.loc[:, ["A57AD2000", "A57AD2010"]]
df_rural = df_rural.apply(pd.to_numeric)

# num of black or african american citizens in Virginia counties over time
df_black = tsdf2.loc[:, ["B18AB2000", "B18AB2010", "B18AB2020"]]
df_black = df_black.apply(pd.to_numeric)

# num of hispanic or latinx citizens in Virginia counties over time
df_latinx = tsdf2.loc[:, ["A35AA2000", "A35AA2010", "A35AA2020"]]
df_latinx = df_latinx.apply(pd.to_numeric)

# num of aapi citizens in Virginia counties over time
df_aapi = tsdf2.loc[:, ["B18AD2000", "B18AD2010", "B18AD2020"]]
df_aapi = df_aapi.apply(pd.to_numeric)

# num of never married citizens in Virginia counties over time
df_marry_male = tsdf2.loc[:, ["BL1AA2000"]]
df_marry_male = df_marry_male.apply(pd.to_numeric)
df_marry_male = df_marry_male.rename(columns={"BL1AA2000": "2000"})
df_marry_female = tsdf2.loc[:, ["BL1AG2000"]]
df_marry_female = df_marry_female.apply(pd.to_numeric)
df_marry_female = df_marry_female.rename(columns={"BL1AG2000": "2000"})
df_marry = df_marry_male.add(df_marry_female)

# num of 65+ y.o. citizens in Virginia counties over time
df_65 = tsdf2.loc[:, ["B57AP2000", "B57AP2010"]]
df_65 = df_65.apply(pd.to_numeric)
df_65 = df_65.rename(columns={"B57AP2000": "2000", "B57AP2010": "2010"})
df_75 = tsdf2.loc[:, ["B57AQ2000", "B57AQ2010"]]
df_75 = df_75.apply(pd.to_numeric)
df_75 = df_75.rename(columns={"B57AQ2000": "2000", "B57AQ2010": "2010"})
df_85 = tsdf2.loc[:, ["B57AR2000", "B57AR2010"]]
df_85 = df_85.apply(pd.to_numeric)
df_85 = df_85.rename(columns={"B57AR2000": "2000", "B57AR2010": "2010"})
df_over_65 = df_65.add(df_75).add(df_85)

# summarize time series for later model
df_avg["Avg College Degree"] = df_edu.mean(axis=1).values
df_avg["Avg Labor Force"] = df_work.mean(axis=1).values
df_avg["Avg Foreigners"] = df_foreign.mean(axis=1).values
df_avg["Avg Females"] = df_female.mean(axis=1).values
df_avg["Avg Income"] = df_income.mean(axis=1).values
df_avg["Avg Poverty"] = df_poverty.mean(axis=1).values
df_avg["Avg Rural"] = df_rural.mean(axis=1).values
df_avg["Avg Black or African American"] = df_black.mean(axis=1).values
df_avg["Avg Latinx"] = df_latinx.mean(axis=1).values
df_avg["Avg AAPI"] = df_aapi.mean(axis=1).values
df_avg["Avg Never Married"] = df_marry.mean(axis=1).values
df_avg["Avg 65+"] = df_over_65.mean(axis=1).values

df_avg

,COUNTY,Avg College Degree,Avg Labor Force,Avg Foreigners,Avg Females,Avg Income,Avg Poverty,Avg Rural,Avg Black or African American,Avg Latinx,Avg AAPI,Avg Never Married,Avg 65+
2845,Accomack County,4098.666667,16189.666667,2137.333333,17892.666667,21745.333333,6535.666667,32060.5,10020.666667,2780.666667,1809.333333,7241.0,6362.5
2846,Albemarle County,33353.000000,48481.333333,8938.333333,49732.000000,37162.666667,7565.000000,41390.5,9067.666667,5299.666667,7444.333333,15104.0,12022.0
2847,Alexandria city,63952.000000,92270.333333,36916.666667,73580.000000,51697.000000,12829.666667,0.0,30436.333333,23592.666667,20833.666667,42714.0,12205.5
2848,Alleghany County,1609.333333,6595.000000,204.666667,7535.666667,23551.000000,1833.666667,8813.5,586.333333,133.666667,93.333333,1625.0,2665.0
2849,Amelia County,1212.666667,6068.666667,165.333333,6184.333333,25031.333333,1198.333333,12045.0,2895.000000,268.666667,166.333333,1989.0,1756.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977,Williamsburg city,3321.666667,6832.000000,969.000000,7400.333333,23749.333333,1759.666667,0.0,3089.333333,819.333333,977.666667,3341.0,1641.5
2978,Winchester city,5069.333333,13769.000000,2854.000000,13267.000000,26168.333333,4170.666667,0.0,2733.000000,3687.333333,2792.333333,6030.0,3553.5
2979,Wise County,3555.333333,15556.666667,423.333333,19589.333333,18154.666667,8299.000000,24113.0,1550.666667,405.000000,297.333333,6862.0,5726.5
2980,Wythe County,3139.000000,14134.333333,163.333333,14670.000000,23034.333333,3737.333333,21393.0,772.000000,264.000000,231.666667,4039.0,4787.5


In [35]:
print('----------- Pre-cleaning | demographic data null counts')
for v in df_avg.columns:
   print(v, sum(df_avg[v].isnull()))

print('\n', df_avg.head(), '\n')

for v in df_avg.columns: # loop through each var
  if v == "COUNTY": continue # skip categorical var
  df_avg[v] = df_avg[v].fillna(np.nanmedian(df_avg[v])) # fill nan with median for that column. Using median omits outliers

print('----------- Post-cleaning | demographic data null counts')
for v in df_avg.columns:
  print(v, sum(df_avg[v].isnull()))

print('\n', df_avg.head())

----------- Pre-cleaning | demographic data null counts
COUNTY 0
Avg College Degree 2
Avg Labor Force 2
Avg Foreigners 2
Avg Females 2
Avg Income 2
Avg Poverty 2
Avg Rural 2
Avg Black or African American 2
Avg Latinx 2
Avg AAPI 2
Avg Never Married 2
Avg 65+ 2

                 COUNTY  Avg College Degree  Avg Labor Force  Avg Foreigners  \
2845   Accomack County         4098.666667     16189.666667     2137.333333   
2846  Albemarle County        33353.000000     48481.333333     8938.333333   
2847   Alexandria city        63952.000000     92270.333333    36916.666667   
2848  Alleghany County         1609.333333      6595.000000      204.666667   
2849     Amelia County         1212.666667      6068.666667      165.333333   

       Avg Females    Avg Income   Avg Poverty  Avg Rural  \
2845  17892.666667  21745.333333   6535.666667    32060.5   
2846  49732.000000  37162.666667   7565.000000    41390.5   
2847  73580.000000  51697.000000  12829.666667        0.0   
2848   7535.666667 

In [36]:
df_avg.loc[df_avg["COUNTY"] == "Nansemond County", "Avg Income"] = np.nanmedian(df_avg["Avg Income"]) # fix null value

In [37]:
vdf

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,11161,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315,TOTAL
1,11162,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL
2,11163,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,RALPH NADER,GREEN,220,11925,20220315,TOTAL
3,11164,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,OTHER,OTHER,261,11925,20220315,TOTAL
4,11165,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315,TOTAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3731,71966,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,OTHER,OTHER,24,12113,20220315,ELECTION DAY
3732,71967,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,OTHER,OTHER,0,12113,20220315,PROVISIONAL
3733,71968,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,2825,12113,20220315,ABSENTEE
3734,71969,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,2370,12113,20220315,ELECTION DAY


In [38]:
df_win["encode_dem"] = df_win["winner"].apply(lambda x: 1 if x == "DEMOCRAT" else 0)
temp_df = df_win.loc[:, ["county", "encode_dem"]].groupby("county").mean()

for v in temp_df.index:
  if v.lower() not in [x.lower().replace("county", "").strip() for x in df_avg["COUNTY"]]:
    temp_df = temp_df.drop(v)

  if "city" not in v.lower():
    temp_df = temp_df.rename(index={v: v + " COUNTY"})

# fix missing values in temp_df
temp_df.loc["BEDFORD CITY"] = temp_df.loc["BEDFORD COUNTY"] # no bedford city in avg df so make same as bedford county
temp_df.loc["NANSEMOND COUNTY"] = 0.5 # give 50/50 chance of voting for democrat for missings since no reference to go off of
temp_df.loc["SOUTH BOSTON CITY"] = 0.5
temp_df.loc["CLIFTON FORGE CITY"] = 0.5
temp_df = temp_df.sort_index()

df_avg["Prob Dem"] = temp_df["encode_dem"].values
df_avg

,COUNTY,Avg College Degree,Avg Labor Force,Avg Foreigners,Avg Females,Avg Income,Avg Poverty,Avg Rural,Avg Black or African American,Avg Latinx,Avg AAPI,Avg Never Married,Avg 65+,Prob Dem
2845,Accomack County,4098.666667,16189.666667,2137.333333,17892.666667,21745.333333,6535.666667,32060.5,10020.666667,2780.666667,1809.333333,7241.0,6362.5,0.000000
2846,Albemarle County,33353.000000,48481.333333,8938.333333,49732.000000,37162.666667,7565.000000,41390.5,9067.666667,5299.666667,7444.333333,15104.0,12022.0,0.833333
2847,Alexandria city,63952.000000,92270.333333,36916.666667,73580.000000,51697.000000,12829.666667,0.0,30436.333333,23592.666667,20833.666667,42714.0,12205.5,1.000000
2848,Alleghany County,1609.333333,6595.000000,204.666667,7535.666667,23551.000000,1833.666667,8813.5,586.333333,133.666667,93.333333,1625.0,2665.0,0.000000
2849,Amelia County,1212.666667,6068.666667,165.333333,6184.333333,25031.333333,1198.333333,12045.0,2895.000000,268.666667,166.333333,1989.0,1756.5,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977,Williamsburg city,3321.666667,6832.000000,969.000000,7400.333333,23749.333333,1759.666667,0.0,3089.333333,819.333333,977.666667,3341.0,1641.5,1.000000
2978,Winchester city,5069.333333,13769.000000,2854.000000,13267.000000,26168.333333,4170.666667,0.0,2733.000000,3687.333333,2792.333333,6030.0,3553.5,1.000000
2979,Wise County,3555.333333,15556.666667,423.333333,19589.333333,18154.666667,8299.000000,24113.0,1550.666667,405.000000,297.333333,6862.0,5726.5,0.000000
2980,Wythe County,3139.000000,14134.333333,163.333333,14670.000000,23034.333333,3737.333333,21393.0,772.000000,264.000000,231.666667,4039.0,4787.5,0.000000


In [39]:
df_avg["Prob Dem"].value_counts()

Prob Dem
0.000000    77
1.000000    36
0.500000     6
0.166667     6
0.333333     5
0.833333     4
0.666667     3
Name: count, dtype: int64

In [40]:
df_avg.to_csv('demographic_info.csv', index=False)